In [23]:
import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="blueberry detection",
    description="blueberry detection pipeline"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="biosens-agrovision-data-volume",
        name="biosens-agrovision-data-volume",
    )
    
    download_component = kfp.components.load_component_from_file('download_model.yaml')
    test_component = kfp.components.load_component_from_file('test_component.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing_model.yaml')
    
    #UNET3
    model_dir = "/mnt/logs/Train_BGFG_BCE_with_weightsUnet3/1_24_01_2023_13_09_38_lr_0.1_step_na_5_epoha_lambda_parametar_1_batch_size_8_sched_multiplicative_loss_bce/"
    model_name = "fully_trained_model_epochs_19_lr_0.1_step_5_Lambda_parametar_1_loss_type_bce_arhitektura_UNet3_batch_size_8.pt"
    model_path = model_dir + model_name

#     #SegNet
#     model_dir = "/mnt/logs/Train_BGFG_BCE_with_weightsUnet3/0_20_03_2023_07_50_46_lr_0.001_step_na_5_epoha_lambda_parametar_1_batch_size_8_sched_multiplicative_loss_bce/"
#     model_name = "fully_trained_model_epochs_19_lr_0.001_step_5_Lambda_parametar_1_loss_type_bce_arhitektura_SegNet_batch_size_8.pt"
#     model_path = model_dir + model_name
    
    
    download_component_task = download_component(
        "biosens-test",
        "/mnt"
        ).add_pvolumes({"/mnt": vop})
#     download_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    test_component_task = test_component(
        model_path,
        "UNet3",
        "cuda",
        download_component_task.output
        ).add_pvolumes({"/mnt": vop})
    test_component_task.set_gpu_limit(1)
#     test_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
#     postprocessing_model = postprocessing_model(
#         "/mnt/DataTest/final_results_folder/test_parcel.tif",
#         "test-result",
#         test_component_task.output
#         ).add_pvolumes({"/mnt": vop}) 
#     postprocessing_model.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    
    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_pipeline.yaml"
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [24]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/AI-platform/kubeflow/Blueberry_row_detection/Test/' + file_name
name = 'Blueberry_test_newUNetOnGPU16'
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

Uploaded pipeline: Blueberry_test_newUNetOnGPU16
